#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
        - object of application based on library called flask
    2. SQLAlchemy db object
        - object of database based on library called SQL


In [244]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///instance/sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User
        - it is the template of user
    - db.Model inheritance
        - inheritance is when the model inherites different values / things from the db
    - _init_ method
        - the constructor of the object
    - ```@property```, ```@<column>.setter```
    - create, read, update, delete methods
        - CRUD is essential things in every database

In [245]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
        - it is similiar to appending the data in lists. But if there is same data, it is unable to append
    2. User Object Constructors
        - who create the more data
    3. Try / Except 
        - try: make an attempt or effort to do something, except: not including; other than


In [246]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Created new uid toby
Created new uid niko
Created new uid lex
Created new uid whit
Created new uid indi
Created new uid raven


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
        - query is a one kind of library and filter means to filter with the conditions
    2. user.password
        - first, other people cannot see the password due to hash. And to verify the identity

In [248]:
# SQLAlchemy extracts single user from database matching User ID
uid = input()
def find_by_uid(uid):
    
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    print(user) # returns user object
find_by_uid(uid)
# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

None


## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
        - make sure that user can find the username and try it and except
    2. user = User(...) This is the equation, user is equal to User(...)
    3. user.dob and try/except dob stands for day of birth
    4. user.create() and try/except create is a similar make

In [249]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
            db.session.commit()
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

None
Created
 {'id': 7, 'name': 'james', 'uid': 'James', 'dob': '09-05-2006', 'age': 16}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all select all of the data from database
    2. json_ready assignment, google List Comprehension return the user in json format

In [250]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-23-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-23-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-23-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7, 'name': 'james', 'uid': 'James', 'dob': '09-05-2006', 'age': 16}]

In [253]:

def update_user():
    
    uid = input("Enter your user id:")
    password = input('Enter your password')
    change = input("what do you want to change")
    if uid is None:
        return {'message': f'User ID is missing'}, 400
    if password is None:
        return {'message': f'password is missing'}, 400
    with app.app_context():
        try:
            user = User.query.filter_by(_uid=uid).first()
        except:
            return "so such user"
        if change == "uid":
            update_content = input("what do you want to change")
            user.uid = update_content
            user_change = User.query.filter_by(_uid=update_content).first()
            db.session.commit()
            print(user_change)
        if change == "password":
            update_content = input("what do you want to change")
            user.password = update_content
            user_change = User.query.filter_by(_password=update_content).first()
            db.session.commit()
            print(user_change)
        if change == "dob":
            update_year = input("Year")
            update_month = input("month")
            update_day = input("day")
            user.dob = datetime(update_year, update_month, update_day)
            db.session.commit()
            print(user)
        

update_user()

{"id": 7, "name": "james", "uid": "James", "dob": "09-05-2006", "age": 16}


In [72]:

def delete_user():
    
    uid = input("Enter your user id:")
    password = input('Enter your password')
    if uid is None:
        return {'message': f'User ID is missing'}, 400
    if password is None:
        return {'message': f'password is missing'}, 400
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    
        user.delete()
        print("Complete delete")
delete_user()
read()

Complete delete


[{'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-15-2023',
  'age': 0}]

In [180]:
import requests

url = "https://billboard3.p.rapidapi.com/hot-100"

querystring = {"date":"2022-07-07","range":"1-10"}

headers = {
	"X-RapidAPI-Key": "56cf0d9c39msh90ab47fd56c02e6p1d2792jsn0f4dfaa46b90",
	"X-RapidAPI-Host": "billboard3.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

songs = response.json()
print(songs[0]['weeksAtNo1'])

[{"artist":"Harry Styles","title":"As It Was","image":"https://charts-static.billboard.com/img/2022/04/harry-styles-bma-as-it-was-po3-180x180.jpg","award":false,"awardsList":[],"rank":1,"weeksAtNo1":8,"lastWeek":2,"change":"up","peakPosition":1,"weeksOnChart":13},{"artist":"Jack Harlow","title":"First Class","image":"https://charts-static.billboard.com/img/2018/01/jack-harlow-i43-180x180.jpg","award":false,"awardsList":[],"rank":2,"lastWeek":3,"change":"up","peakPosition":1,"weeksOnChart":12},{"artist":"Lizzo","title":"About Damn Time","image":"https://charts-static.billboard.com/img/2022/04/lizzo-o9x-about-damn-time-f6q-180x180.jpg","award":true,"awardsList":[{"awardName":"Gains In Performance","svg":"<svg xmlns=\"http://www.w3.org/2000/svg\" style=\"width:100%;height:auto\" viewBox=\"0 0 128 171.211\"><g data-name=\"Group 12\"><path data-name=\"Path 3036\" d=\"M0 87.211v-24l64-32 64 32v24l-64-32z\" fill=\"#fcee21\"></path><path data-name=\"Path 3037\" d=\"M0 127.211v-24l64-32 64 32v2

In [255]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///instance/songs.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

In [256]:

from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError

class Song(db.Model):
    __tablename__ = 'billboard'  # table name is plural, class name is singular
    
    
    _rank = db.Column(db.Integer, primary_key=True)
    _title= db.Column(db.String(255), unique=True, nullable=False)
    _artist = db.Column(db.String(255), unique=False, nullable=False)
    _lastWeek = db.Column(db.String(255), unique=False, nullable=False)
    _peakPosition = db.Column(db.String(255), unique=False, nullable=False)
    _weeksOnChart = db.Column(db.String(255), unique=False, nullable=False)
   

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, rank, title, artist, lastWeek, peakPosition, weeksOnChart):
        self._rank = rank
        self._title =  title  # variables with self prefix become part of the object, 
        self._artist = artist
        self._lastWeek = lastWeek
        self._peakPosition = peakPosition
        self._weeksOnChart = weeksOnChart
     

    @property
    def rank(self):
        return self._rank
    
    # a setter function, allows name to be updated after initial object creation
    @rank.setter
    def rank(self, rank):
        self._rank = rank

    # a name getter method, extracts name from object
    @property
    def title(self):
        return self._title
    
    # a setter function, allows name to be updated after initial object creation
    @title.setter
    def title(self, title):
        self._title = title
    
    # a getter method, extracts uid from object
    @property
    def artist(self):
        return self._artist
    
    # a setter function, allows uid to be updated after initial object creation
    @artist.setter
    def artist(self, artist):
        self._artist = artist
        



    # check password parameter against stored/encrypted password
    # def is_subject(self, subject):
    #     if subject == "English" | subject == "Math" | subject == "Foreign Language" | subject == "Elective" | subject == "Civics" | subject == "Economies" | subject == "History" | subject == "Science":
    #     return result
    
    
    @property
    def lastWeek(self):
        return self._lastWeek
    
    @lastWeek.setter
    def lastWeek(self, lastWeek):
        self._lastWeek = lastWeek
    
    @property
    def peakPosition(self):
        return self._peakPosition
    
    @peakPosition.setter
    def peakPosition(self, peakPosition):
        self._peakPosition = peakPosition

    @property
    def weeksOnChart(self):
        return self._weeksOnChart
    
    @weeksOnChart.setter
    def weeksOnChart(self, weeksOnChart):
        self._weeksOnChart = weeksOnChart
    
  
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "rank": self.rank,
            "title": self.title,
            "artist": self.artist,
            "last week": self.lastWeek,
            "peakPosition": self.peakPosition,
            "weeksOnChart": self.weeksOnChart
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, title="", artist = "", lastWeek = "", peakPosition = "", weeksOnChart = ""):
        """only updates values with length"""
        if len(title) > 0:
            self.title = title
        if len(artist) > 0:
            self.artist = artist
        
        if len(lastWeek) > 0:
            self.lastWeek = lastWeek
        if len(peakPosition) > 0:
            self.peakPosition = peakPosition
        if len(weeksOnChart) > 0:
            self.weeksOnChart = weeksOnChart
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

In [227]:
import requests
def initSongs():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        songss = []


        url = "https://billboard3.p.rapidapi.com/hot-100"

        querystring = {"date":"2022-07-07","range":"1-10"}

        headers = {
            "X-RapidAPI-Key": "56cf0d9c39msh90ab47fd56c02e6p1d2792jsn0f4dfaa46b90",
            "X-RapidAPI-Host": "billboard3.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)



        songs = response.json()
       
        for i in range(len(songs)):

            
            songss.append(Song(rank = int(songs[i]['rank']), title = songs[i]['title'], artist = songs[i]['artist'], lastWeek=songs[i]['lastWeek'], peakPosition= songs[i]['peakPosition'], weeksOnChart=songs[i]['weeksOnChart']))

        for song in songss:
            try:
          
            

                object = song.create()
                print(f"Created new uid {object}")
                
            except:  # error raised if object nit created
            
                print(f"Records exist or error.")
                
initSongs()

Created new uid {"rank": 1, "title": "As It Was", "artist": "Harry Styles", "last week": "2", "peakPosition": "1", "weeksOnChart": "13"}
Created new uid {"rank": 2, "title": "First Class", "artist": "Jack Harlow", "last week": "3", "peakPosition": "1", "weeksOnChart": "12"}
Created new uid {"rank": 3, "title": "About Damn Time", "artist": "Lizzo", "last week": "5", "peakPosition": "3", "weeksOnChart": "11"}
Created new uid {"rank": 4, "title": "Wait For U", "artist": "Future Featuring Drake & Tems", "last week": "4", "peakPosition": "1", "weeksOnChart": "9"}
Created new uid {"rank": 5, "title": "Jimmy Cooks", "artist": "Drake Featuring 21 Savage", "last week": "1", "peakPosition": "1", "weeksOnChart": "2"}
Created new uid {"rank": 6, "title": "Running Up That Hill (A Deal With God)", "artist": "Kate Bush", "last week": "9", "peakPosition": "4", "weeksOnChart": "25"}
Created new uid {"rank": 7, "title": "Break My Soul", "artist": "Beyonce", "last week": "15", "peakPosition": "7", "weeks

In [240]:
import pandas as pd
def read_songs():
    with app.app_context():
        songs = Song.query.all()
        json_ready = [song.read() for song in songs]
        
        test_result = pd.DataFrame(json_ready)
        print(test_result)
read_songs()

   rank                                   title  \
0     1                             First Class   
1     2                         About Damn Time   
2     3                              Wait For U   
3     4                             Jimmy Cooks   
4     5  Running Up That Hill (A Deal With God)   
5     6                           Break My Soul   
6     7                         Me Porto Bonito   
7     8                              Heat Waves   
8     9                           Glimpse Of Us   

                          artist last week peakPosition weeksOnChart  
0                    Jack Harlow         3            1           12  
1                          Lizzo         5            3           11  
2  Future Featuring Drake & Tems         4            1            9  
3      Drake Featuring 21 Savage         1            1            2  
4                      Kate Bush         9            4           25  
5                        Beyonce        15            7        

In [229]:
def delete_songs():
    
    title = input("Enter your user id:")
    print(title)
    if title is None:
        return {'message': f'User ID is missing'}, 400
    with app.app_context():
        user = Song.query.filter_by(_title=title).first()
 
        rank = int(user.rank)
        user.delete()
        print("Complete delete")
        i = 1
        while True:
            if int(rank + i) > 10:
                break
            user = Song.query.filter_by(_rank=rank+i).first()
            if user is None:
                print("not found")
                break
            rank_number = user.read()['rank']
            b = rank_number - 1
            user.rank = b
            i += 1
            db.session.commit()
delete_songs()
with app.app_context():
    songs = Song.query.all()
    json_ready = [song.read() for song in songs]
print(json_ready)

Complete delete


In [257]:
def update_songs(): 
    title = input("Enter your user id:")
    print(title)
    if title is None:
        return {'message': f'User ID is missing'}, 400
    with app.app_context():
        user = Song.query.filter_by(_title=title).first()
        art = input("please change artist")
        if user is None:
            print("not found")
        user.artist = art
        db.session.commit()
        user1 = Song.query.filter_by(_title=title).first()
    print(user1)
update_songs()

First Class
{"rank": 1, "title": "First Class", "artist": "James", "last week": "3", "peakPosition": "1", "weeksOnChart": "12"}


# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.